In [ ]:
%use dataframe, kandy

# Getting historical data

In [ ]:
import kotlin.random.Random

data class DieRoll(val roll: Int, val result: Int)
fun rollTheDice() = Random.nextInt(1, 7)
//Imagine we have rolled a 6-sided die, 100 times
val historicalDiceRolls = generateSequence { rollTheDice() }.take(100).mapIndexed { idx, it -> DieRoll(idx+1, it) }.toList().toDataFrame()
historicalDiceRolls

# Use MonteCarlo to simulate a probability distribution
We want to know what the sum of the dice will be, given a certain amount of dice rolls.
We understand that it's not always going to be the same, but let's say that we're happy with 50% certainty.

In [ ]:
data class Trial(val totalDiceScore: Int)

fun monteCarloFromDiceRolls(
    trials: Int = 10_000,
    drawsPerTrial: Int = 30,
): DataFrame<Trial> {
    val values = historicalDiceRolls.get { result }.toList()
    val n = values.size
    val totals = List(trials) { //create 10_000 trials
        var currentTotal = 0
        repeat(drawsPerTrial) { //roll 30 times and pick a result from our historical rolls to do so
            val idx = Random.nextInt(n)
            currentTotal += values[idx]
        }
        Trial(currentTotal)
    }
    return totals.toDataFrame()
}

In [ ]:
val mc = monteCarloFromDiceRolls(
    trials = 10_000,
    drawsPerTrial = 30,
)
mc

# Calculate some percentiles
`p50` = the 50th percentile, or in this case: If we're ok with being 50% sure, what's a total score I can get having rolled 30 dice.
`p85` = the 85th percentile

In [ ]:
val p50 = mc.percentile(50.0) { totalDiceScore } //aka median
val p85 = mc.percentile(85.0) { totalDiceScore }
println("P50=$p50  P85=$p85")

# Prep the distribution for visualisation
We'll want to show how many times (frequency) a totalDiceScore was gotten with those 30 rolls.
So we'll need to group and count them.
The groups will need to be sorted for the visualisation to make sense.

In [ ]:
val frequencyPerTotal = mc.groupBy { totalDiceScore }.count().rename("count" to "frequency").sortBy { totalDiceScore }
frequencyPerTotal

In [ ]:
import org.jetbrains.kotlinx.statistics.distribution.NormalDistribution
import org.jetbrains.letsPlot.Stat
import org.jetbrains.letsPlot.core.spec.plotson.BinStatOptions

val distributionBarChart = plot(frequencyPerTotal) {
    layout { size = 2000 to 900 }
    bars {
        x(totalDiceScore)
        y(frequency)
        alpha = 0.6
    }
    // vertical markers
    vLine { xIntercept.constant(p50); type = LineType.DASHED }
    vLine { xIntercept.constant(p85); type = LineType.DASHED }
}
distributionBarChart